## Initialization

First load all required packages:

In [3]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using JuMP, Juniper, Cbc, Ipopt

Activating environment at `~/github/simple-minlp-example/Project.toml`


## Generate Instance Data

In [ ]:
d = 10 
max_val = 10000
x = rand(1:max_val, d)
y = rand(1:max_val, d)
B = rand(d, d)

## Fixed Matrix B

Simple convex quadratic non-linear programming model:

In [4]:
function get_A(x, y, B, nneg = false)
    d = length(x)
    model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))
    if nneg
        @variable(model, A[1:d, 1:d] >=0 )
    else
        @variable(model, A[1:d, 1:d])
    end
    @constraint(model, y .== A*x)
    @NLobjective(model, Min, sum((A[i,j] - B[i,j])^2 for i in 1:d, j in 1:d))
    optimize!(model)
    value.(A)
end

get_A (generic function with 2 methods)

Get unconstrained `A`:

In [29]:
A = get_A(x, y, B);
display(A)

10×10 Array{Float64,2}:
 -0.0144436    -0.0662513   -0.00270775   …  -0.0192127    -0.000510254
  0.0176725     0.081062     0.00331307       0.0235078     0.000624322
  0.0104997     0.0481611    1.00197          0.0139666     0.000370927
 -0.0149048    -0.0683671   -0.00279422      -0.0198263    -0.000526549
  1.00663       0.0303883    0.001242         0.00881255    0.000234044
  0.00315395    0.0144669    0.000591272  …   1.0042        0.000111421
  0.000282529   0.00129593   5.29659e-5       0.000375818   9.98101e-6 
 -0.000184509   0.999154    -3.459e-5        -0.000245432  -6.51821e-6 
  0.00695367    0.0318958    0.00130361       0.00924973    0.000245655
  0.0103844     0.0476322    0.00194677       0.0138132     1.00037    

Get non-negative `A`:

In [33]:
A_nneg = get_A(x, y, B, true);
display(A_nneg)

10×10 Array{Float64,2}:
 9.16432e-9   0.0         9.2225e-8    …  4.40717e-9   5.32353e-7 
 0.0176725    0.081062    0.00331345      0.0235079    0.000626337
 0.0104998    0.0481611   1.00197         0.0139667    0.000374891
 9.33671e-9   0.0         9.31445e-8      4.53677e-9   5.37229e-7 
 1.00662      0.0303883   0.001243        0.00881267   0.000243166
 0.00315429   0.0144667   0.000593399  …  1.0042       0.000137354
 0.000288097  0.00129414  9.5878e-5       0.000378711  7.08208e-5 
 1.75001e-6   0.99671     2.13439e-5      1.30458e-6   5.39195e-5 
 0.00695383   0.0318958   0.00130457      0.00924984   0.000254012
 0.0103845    0.0476322   0.00194741      0.0138133    1.00037    

## Variable Permutation Matrix B

Simple mixed-integer convex quadratic non-linear programming model:

In [27]:
function get_A_perm(x, y, nneg = false)
    optimizer = Juniper.Optimizer
    params = Dict{Symbol,Any}()
    params[:nl_solver] = with_optimizer(Ipopt.Optimizer, print_level=0)
    params[:mip_solver] = with_optimizer(Cbc.Optimizer, logLevel=0)
    params[:log_levels] = Symbol[]
    d = length(x)
    model = Model(with_optimizer(optimizer, params))
    if nneg
        @variable(model, A[1:d, 1:d] >=0 )
    else
        @variable(model, A[1:d, 1:d])
    end
    @variable(model, B[1:d, 1:d], Bin)
    @constraint(model, [i = 1:d], sum(B[:,i]) == 1)
    @constraint(model, [i = 1:d], sum(B[i,:]) == 1)
    @constraint(model, y .== A*x)
    @NLobjective(model, Min, sum((A[i,j] - B[i,j])^2 for i in 1:d, j in 1:d))
    optimize!(model)
    value.(A), round.(value.(B))
end

get_A_perm (generic function with 2 methods)

Get unconstrained `A` and permutation matrix `B`:

In [34]:
A_perm, B_perm = get_A_perm(x, y);
display(A_perm)
display(B_perm)

10×10 Array{Float64,2}:
 -0.0144436    -0.0662513   -0.00270775   …  -0.0192127    -0.000510254
  0.0176725     0.081062     0.00331307       0.0235078     0.000624322
  0.0104997     0.0481611    1.00197          0.0139666     0.000370927
 -0.0149048    -0.0683671   -0.00279422      -0.0198263    -0.000526549
  1.00663       0.0303883    0.001242         0.00881256    0.000234044
  0.00315394    0.0144669    0.000591272  …   1.0042        0.000111421
  0.000282529   0.00129593   5.29659e-5       0.000375818   9.98101e-6 
 -0.000184509   0.999154    -3.459e-5        -0.000245432  -6.51821e-6 
  0.00695367    0.0318958    0.00130361       0.00924973    0.000245655
  0.0103844     0.0476322    0.00194678       0.0138132     1.00037    

10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

Get non-negative `A` and permutation matrix `B`:

In [35]:
A_perm_nneg, B_perm_nneg = get_A_perm(x, y, true);
display(A_perm_nneg)
display(B_perm_nneg)

10×10 Array{Float64,2}:
 0.0          0.0         2.70854e-8   …  0.0          1.86784e-7 
 0.0176725    0.081062    0.00331321      0.0235078    0.000625052
 0.0104998    0.0481611   1.00197         0.0139667    0.000372285
 0.0          0.0         2.7419e-8       0.0          1.88554e-7 
 1.00663      0.0303883   0.00124236      0.0088126    0.000237318
 0.00315407   0.0144668   0.000592041  …  1.0042       0.000124588
 0.000284375  0.00129509  7.97996e-5      0.000376753  5.35849e-5 
 6.27034e-7   0.996712    9.98504e-6      4.67399e-7   3.6919e-5  
 0.00695373   0.0318958   0.00130398      0.00924977   0.000248602
 0.0103844    0.0476322   0.001947        0.0138133    1.00037    

10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0